In [194]:
import tushare as ts
from datetime import date
from datetime import timedelta 
import numpy as np
import pandas as pd
from scipy import stats
import numpy_financial as npf
import pandas_datareader.data as web

In [195]:
pro = ts.pro_api('41cbc98ddd6c55a141cf281d0636c4ef25c13ff69e38bc400f3f4ebd')

In [67]:
company = '600519.SH' # Kweichow Moutai Co Ltd (replaceable)
start_date = '20181231' # (replaceable)
end_date = '20191231' # (replaceable)

Estimate Future Revenue Growth:

In [196]:
revenue_1 = pro.income(ts_code=company, period=end_date, fields='revenue')['revenue'][0]
revenue_2 = pro.income(ts_code=company, period=start_date, fields='revenue')['revenue'][0]
revenue_g = (revenue_1 - revenue_2) /revenue_2
print(revenue_g)

0.16011517690127722


Estimate Future FCFF:

In [100]:
# FCFF = Operating Cash Flow - Capital Expenditure + Interest Expense * (1 - tax)
oper_cashflow = pro.cashflow(ts_code=company, period=end_date, fields='n_cashflow_act')['n_cashflow_act'][0]
cap_ex =pro.cashflow(ts_code=company, period=end_date, fields='c_pay_acq_const_fiolta')['c_pay_acq_const_fiolta'][0]
int_ex = pro.income(ts_code=company, period=end_date, fields='int_exp')['int_exp'][0]
gross_pro = pro.income(ts_code=company, period=end_date, fields='total_profit')['total_profit'][0]
net_pro = pro.income(ts_code=company, period=end_date, fields='n_income')['n_income'][0]
taxt_rate = net_pro / gross_pro
fcff = oper_cashflow - cap_ex + int_ex * taxt_rate
# Future FCFF = Current FCFF * Revenue Growth (Can be estimated in more SPECIFIC way)
future_fcff = pd.Series(0, index = [0, 1, 2, 3, 4])
future_fcff[0] = fcff * (1 + revenue_g)
for i in range(1, 5):
    future_fcff[i] = future_fcff[i - 1] * (1 + revenue_g)
print(future_fcff)

0    48922953432
1    56756260775
2    65843799509
3    76386391115
4    88617011641
dtype: int64


Calculate Cost of Debt

In [197]:
ebit = pro.income(ts_code=company, period=end_date, fields='ebit')['ebit'][0]
interest_expense = pro.income(ts_code=company, period=end_date, fields='int_exp')['int_exp'][0]
interest_coverage_ratio = ebit / interest_expense

yesterday = date.today()- timedelta(days = 60)
today = date.today()
start = yesterday.strftime('%Y-%m-%d')
end = today.strftime('%Y-%m-%d')
Treasury = web.DataReader(['TB1YR'], 'fred', start, end)
rf = float(Treasury.iloc[-1]) 

if interest_coverage_ratio > 8.5:
    #Rating is AAA
    credit_spread = 0.0063
if (interest_coverage_ratio > 6.5) & (interest_coverage_ratio <= 8.5):
    #Rating is AA
    credit_spread = 0.0078
if (interest_coverage_ratio > 5.5) & (interest_coverage_ratio <=  6.5):
    #Rating is A+
    credit_spread = 0.0098
if (interest_coverage_ratio > 4.25) & (interest_coverage_ratio <=  5.49):
    #Rating is A
    credit_spread = 0.0108
if (interest_coverage_ratio > 3) & (interest_coverage_ratio <=  4.25):
    #Rating is A-
    credit_spread = 0.0122
if (interest_coverage_ratio > 2.5) & (interest_coverage_ratio <=  3):
    #Rating is BBB
    credit_spread = 0.0156
if (interest_coverage_ratio > 2.25) & (interest_coverage_ratio <=  2.5):
    #Rating is BB+
    credit_spread = 0.02
if (interest_coverage_ratio > 2) & (interest_coverage_ratio <=  2.25):
    #Rating is BB
    credit_spread = 0.0240
if (interest_coverage_ratio > 1.75) & (interest_coverage_ratio <=  2):
    #Rating is B+
    credit_spread = 0.0351
if (interest_coverage_ratio > 1.5) & (interest_coverage_ratio <=  1.75):
    #Rating is B
    credit_spread = 0.0421
if (interest_coverage_ratio > 1.25) & (interest_coverage_ratio <=  1.5):
    #Rating is B-
    credit_spread = 0.0515
if (interest_coverage_ratio > 0.8) & (interest_coverage_ratio <=  1.25):
    #Rating is CCC
    credit_spread = 0.0820
if (interest_coverage_ratio > 0.65) & (interest_coverage_ratio <=  0.8):
    #Rating is CC
    credit_spread = 0.0864
if (interest_coverage_ratio > 0.2) & (interest_coverage_ratio <=  0.65):
    #Rating is C
    credit_spread = 0.1134
if interest_coverage_ratio <=  0.2:
    #Rating is D
    credit_spread = 0.1512
    
cost_of_debt = rf + credit_spread
print(cost_of_debt)

0.1363


Calculate Cost of Equity (CAPM)

In [198]:
# estimate beta with 399300.SZ
price = pro.daily(ts_code=company, start_date=start_date, end_date=end_date)['close']
stock_return = price.pct_change()[1:]
benchmark = pro.index_daily(ts_code='399300.SZ', start_date=start_date, end_date=end_date)['close']
market_return = benchmark.pct_change()[1:]
beta = stats.linregress(market_return, stock_return)[0]
benchmark_past = pro.index_daily(ts_code='399300.SZ', start_date=start_date, end_date=end_date)['close'][243]
benchmark_current = pro.index_daily(ts_code='399300.SZ', start_date=start_date, end_date=end_date)['close'][0]
market_year_return = benchmark_current / benchmark_past - 1
cost_of_equity = rf +( beta * ( market_year_return - rf))
print(cost_of_equity)

0.3846287462068252


Calculate WACC

In [199]:
debt = pro.balancesheet(ts_code=company, period=end_date,fields='total_liab')['total_liab'][0]
equity = pro.balancesheet(ts_code=company, period=end_date,fields='total_hldr_eqy_inc_min_int')['total_hldr_eqy_inc_min_int'][0]
debt_percent = debt / (debt + equity)
equity_percent = equity / (debt + equity)
wacc = (cost_of_debt * (1 - taxt_rate) * debt_percent) + (cost_of_equity * equity_percent)
print(wacc)

0.3058506311955349


 Get the Net Present Value of the FCFF

In [200]:
future_fcff_List = future_fcff.tolist()
NPV = npf.npv(wacc,future_fcff_List)
print(NPV)

195776729681.80984


Calculate the Terminal Value

In [201]:
Terminal_Value = (future_fcff[4] * (1 + revenue_g)) /(wacc  - revenue_g)
Terminal_Value_Discounted = Terminal_Value /(1 + wacc ) ** 5
print(Terminal_Value_Discounted)

185774231446.16635


Get the Company Valuation

In [204]:
total_value = NPV + Terminal_Value_Discounted
equity_value = total_value - debt
total_share = pro.daily_basic(ts_code=company, trade_date=end_date, fields='total_share')['total_share'][0] * 10000
estimate_price = equity_value / total_share
print(estimate_price)

270.96446858458614


Compare with Real Price

In [206]:
real_price = pro.daily_basic(ts_code=company, trade_date=end_date, fields='close')['close'][0]
if (estimate_price < real_price):
    print("The company is overestimated at " , real_price)
else:
    print("The company is underestimated at " , real_price)

The company is overestimated at  1183.0


2. comparable analysis

Select comparable companies by industry, size, location, etc.

In [168]:
company = '600519.SH' # Kweichow Moutai Co Ltd (replaceable)
compare1 = "000858.SZ" # Wuliangye Yibin Co Ltd (replaceable)
compare2 = "000568.SZ" # Luzhou Laojiao Co Ltd (replaceable)
compare3 = "002304.SZ" # Jiangsu Yanghe Brewery Joint-Stock Co Ltd (replaceable)
compare4 = "600809.SH" # Shanxi Xinghuacun Fen Wine Factory Co Ltd (replaceable)
compare5 = "603589.SH" # Anhui Kouzi Distillery Co Ltd (replaceable)
compare = pd.Series([compare1, compare2, compare3, compare4, compare5], index = [0, 1, 2, 3, 4])
trade_date = '20200918' # (replaceable)

Calculate Median and Average of EV/EBITDA, PE and PS

In [189]:
evEbitda = pd.Series(0, index = [0, 1, 2, 3, 4])
pe = pd.Series(0, index = [0, 1, 2, 3, 4])
ps = pd.Series(0, index = [0, 1, 2, 3, 4])
for i in range(0, 4):
    mv = pro.daily_basic(ts_code=compare[i], trade_date=trade_date, fields='total_mv')['total_mv'][0] * 10000 
    debt = pro.balancesheet(ts_code=compare[i], period=end_date,fields='total_liab')['total_liab'][0]
    cash = pro.cashflow(ts_code=compare[i], trade_date=end_date, fields='c_cash_equ_end_period')['c_cash_equ_end_period'][0]
    ebitda = pro.income(ts_code=compare[i], period=end_date,fields='ebitda')['ebitda'][0]
    ev = mv + debt - cash
    evEbitda[i] = ev / ebitda 
    pe[i] = pro.daily_basic(ts_code=compare[i], trade_date=trade_date, fields='pe_ttm')['pe_ttm'][0]
    ps[i] = pro.daily_basic(ts_code=compare[i], trade_date=trade_date, fields='ps_ttm')['ps_ttm'][0]
print("The median of EV/EBITDA is ", evEbitda.median()," and the average of EV/EBITDA is ", evEbitda.mean())
print("The median of PE is ", pe.median()," and the average of PE is ", pe.mean())
print("The median of PS is ", ps.median()," and the average of PS is ", ps.mean())

The median of EV/EBITDA is  22.42403790085872  and the average of EV/EBITDA is  19.808012633619573
The median of PE is  40.0  and the average of PE is  37.2
The median of PE is  13.0  and the average of PE is  10.2


Compare with the metrics of the target company

In [192]:
company_mv = pro.daily_basic(ts_code=company, trade_date=trade_date, fields='total_mv')['total_mv'][0] * 10000 
company_debt = pro.balancesheet(ts_code=company, period=end_date,fields='total_liab')['total_liab'][0]
company_cash = pro.cashflow(ts_code=company, trade_date=end_date, fields='c_cash_equ_end_period')['c_cash_equ_end_period'][0]
company_ebitda = pro.income(ts_code=company, period=end_date,fields='ebitda')['ebitda'][0]
company_ev = company_mv + company_debt - company_cash
company_evEbitda = company_ev / company_ebitda 
company_pe = pro.daily_basic(ts_code=company, trade_date=trade_date, fields='pe_ttm')['pe_ttm'][0]
company_ps = pro.daily_basic(ts_code=company, trade_date=trade_date, fields='ps_ttm')['ps_ttm'][0]
print("The EV/EBITDA of the company is ", company_evEbitda)
print("The PE of the company is ", company_pe)
print("The PS of the company is ", company_ps)

The EV/EBITDA of the company is  34.18077065097994
The PE of the company is  48.5498
The PS of the company is  23.6861


The metrics of target company is noticeably higher than the median and average, which indicates it may be overestimated.

Reference:
https://codingandfun.com/calculating-weighted-average-cost-of-capital-wacc-with-python/
https://www.codingfinance.com/post/2018-04-25-portfolio-beta-py/
https://corporatefinanceinstitute.com/resources/knowledge/valuation/comparable-company-analysis/